In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.icu_interface import Inpatients

In [3]:
import logging
logging.root.level = logging.DEBUG


In [ ]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR):
    m4inpatient_dataset = load_dataset('M4ICU')
   

DEBUG:root:Loading dataframes


In [ ]:
splits = m4inpatient_dataset.random_splits(0.8, 0.9, 42)

In [ ]:
list(map(len, splits))

In [ ]:
import jax
from concurrent.futures import ThreadPoolExecutor

jax.config.update('jax_platform_name', 'cpu')

In [ ]:
%%timeit
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:2048], max_workers=1)
m4inaptients_jax = m4inaptients.to_jax_arrays()

In [ ]:
%%timeit
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:2048], max_workers=2)
m4inaptients_jax = m4inaptients.to_jax_arrays()

In [ ]:
%%timeit
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:2048], max_workers=4)
m4inaptients_jax = m4inaptients.to_jax_arrays()

In [ ]:
%%timeit
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:2048], max_workers=8)
m4inaptients_jax = m4inaptients.to_jax_arrays()

In [ ]:
%%timeit
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:2048], max_workers=16)
m4inaptients_jax = m4inaptients.to_jax_arrays()

In [ ]:
m4inaptients.size_in_bytes / 1024 ** 3

In [ ]:
import jax.tree_util as jtree
import equinox as eqx
import jax.numpy as jnp
is_arr = eqx.filter(m4inaptients.subjects, eqx.is_array)
arr_size = jtree.tree_map(lambda a, flag: jnp.size(a) * jnp.dtype(a).itemsize if flag is not None else 0, 
                         m4inaptients.subjects, is_arr)
batch_size = sum(jtree.tree_leaves(arr_size))
batch_size
# is_arr = eqx.filter(m4inaptients.subjects, eqx.is_array)
# jtree.tree_leaves(m4inaptients.subjects)

In [ ]:
is_arr[0].admissions[0].procedures

In [ ]:
m4inaptients.subjects[0].admissions[0].procedures

In [ ]:
eqx.is_array([0, 1])

In [ ]:
import jax.numpy as jnp
a = jnp.array([1, 1, 0], dtype=jnp.float16)
jnp.dtype(a).itemsize * jnp.size(a)


In [ ]:
a.size

In [ ]:
from lib.ehr.coding_scheme import DxICD9, DxICD10

In [ ]:
s = DxICD10()

In [ ]:
len(s)

## TODO

1. Squeeze code vectors.
2. Squeeze float32 to float16.

In [ ]:
!ls  /home/asem/GP/ICE-NODE/lib/ehr/resources/

In [ ]:
df = pd.read_csv('/home/asem/GP/ICE-NODE/lib/ehr/resources/mimic4_int_grouper_proc.csv.gz')

In [ ]:
df.group.nunique()